In [1]:
# MAE: 528.190
# XGBoostRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import xgboost as xgb

train = pd.read_csv('../Data/used_car_train_20200313.csv', sep=' ')
test = pd.read_csv('../Data/used_car_testB_20200421.csv', sep=' ')

# 合并训练数据和测试数据集
all_data = pd.concat([train, test], ignore_index=True)

# 对 price 做对数变换
all_data['price'] = np.log1p(all_data['price'])

In [2]:
# 处理异常值，如功率大于 600 的值
all_data['power'] = all_data['power'].apply(lambda x: 600 if x > 600 else x)

# 处理日期相关信息
all_data['reg_year'] = all_data['regDate'].apply(lambda x: int(str(x)[:4]))
all_data['reg_month'] = all_data['regDate'].apply(lambda x: int(str(x)[4:6]))
all_data['reg_day'] = all_data['regDate'].apply(lambda x: int(str(x)[6:]))
all_data['creat_year'] = all_data['creatDate'].apply(lambda x: int(str(x)[:4]))
all_data['creat_month'] = all_data['creatDate'].apply(lambda x: int(str(x)[4:6]))
all_data['creat_day'] = all_data['creatDate'].apply(lambda x: int(str(x)[6:]))

# 标记汽车没有经过维修
all_data['notRepairedDamage'] = all_data['notRepairedDamage'].apply(lambda x: 0 if x == '-' else 1)

# 对可分类的连续特征进行分桶，如将功率（power）分成10个分桶，并提取新特征
all_data['power_bucket'] = pd.cut(all_data['power'], 10, labels=False)
new_cols = ['power_bucket', 'v_0', 'v_3', 'v_8', 'v_12']
for col1 in new_cols:
    for col2 in new_cols:
        if col1 != col2:
            all_data['{}_{}_sum'.format(col1, col2)] = all_data[col1] + all_data[col2]
            all_data['{}_{}_diff'.format(col1, col2)] = all_data[col1] - all_data[col2]

# 处理缺失值
all_data['fuelType'] = all_data['fuelType'].fillna(0)
all_data['gearbox'] = all_data['gearbox'].fillna(0)
all_data['bodyType'] = all_data['bodyType'].fillna(0)
all_data['model'] = all_data['model'].fillna(0)

# 分离特征和标签
train_data = all_data[~all_data['price'].isnull()]
test_data = all_data[all_data['price'].isnull()]
X_train = train_data.drop(['SaleID', 'name', 'regDate', 'creatDate', 'price'], axis=1)
X_test = test_data.drop(['SaleID', 'name', 'regDate', 'creatDate', 'price'], axis=1)
y_train = train_data['price']

In [3]:
# 定义模型参数
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

params = {
    'max_depth': 10,
    'learning_rate': 0.05,
    'num_boost_round': 1000,  # 注意：这个参数在 xgb.train() 中应改为 num_boost_round
    'gamma': 0.005,
    'subsample': 0.9,
    'colsample_bytree': 0.7,
    'objective': 'reg:squarederror',
    'random_state': 2021,
    'eval_metric': 'mae'
}

skf = KFold(n_splits=5, shuffle=True, random_state=2021)
oof = np.zeros(len(X_train))
test_predict = np.zeros(len(X_test))

for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
    print("Training on Fold {}".format(i+1))
    tr_x, tr_y = X_train.iloc[train_index], y_train.iloc[train_index]
    vl_x, vl_y = X_train.iloc[val_index], y_train.iloc[val_index]

    dtrain_fold = xgb.DMatrix(tr_x, label=tr_y)
    dval_fold = xgb.DMatrix(vl_x, label=vl_y)

    watchlist = [(dtrain_fold, 'train'), (dval_fold, 'eval')]
    num_round = 1000  # 你可以设置一个较大的值，早停会提前停止
    bst_model = xgb.train(params, dtrain_fold, num_round, watchlist, early_stopping_rounds=100, verbose_eval=200)

    oof[val_index] = bst_model.predict(dval_fold)
    test_predict += bst_model.predict(dtest) / skf.n_splits

mae = mean_absolute_error(np.expm1(y_train), np.expm1(oof))

print("MAE: {:.3f}".format(mae))

# 保存预测结果
submission = pd.DataFrame({'SaleID': test_data['SaleID'], 'price': np.expm1(test_predict)})

submission.to_csv('xgb_submission.csv', index=False)


Training on Fold 1


D:\python\Lib\site-packages\xgboost\core.py:723: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
D:\python\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:55:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mae:0.94671	eval-mae:0.94409
[200]	train-mae:0.08540	eval-mae:0.12823


KeyboardInterrupt: 